In [ ]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import librosa
from PIL import Image
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from transformers import ConvNextForImageClassification
from torch.optim import AdamW
from sklearn.metrics import roc_curve, roc_auc_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


CUDA available: True
GPU name: NVIDIA GeForce RTX 4090 D


In [3]:


LABEL_MAP = {"bonafide": 1, "spoof": 0}

class ConvNextFlacDataset(Dataset):
    def __init__(self, csv_path, audio_dir, nrows=None):
        self.df = pd.read_csv(csv_path, sep=" ", header=None, nrows=nrows)
        self.audio_dir = audio_dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        audio_filename = row[1] + ".flac"
        label_str = row[4]
        label = LABEL_MAP[label_str]

        flac_path = os.path.join(self.audio_dir, audio_filename)
        waveform, sr = torchaudio.load(flac_path)

        # Convert to mono
        waveform = waveform.mean(dim=0)

        # Resample to 16kHz if needed
        target_sr = 16000
        if sr != target_sr:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
            waveform = resampler(waveform)

        # Convert to numpy for librosa
        waveform_np = waveform.numpy()

        # Compute spectrogram
        spec = librosa.stft(waveform_np, n_fft=512, hop_length=160, win_length=400)
        mag, _ = librosa.magphase(spec)
        spec_db = librosa.amplitude_to_db(mag, ref=np.max)

        # Normalize to 0-255 and convert to uint8
        spec_norm = (spec_db - spec_db.min()) / (spec_db.max() - spec_db.min() + 1e-9) * 255.0
        spec_uint8 = spec_norm.astype(np.uint8)  # shape: [freq, time]

        # Truncate or pad to 224x224
        H, W = spec_uint8.shape
        target_H = min(224, H)
        target_W = min(224, W)
        start_H = (H - target_H) // 2
        start_W = (W - target_W) // 2
        spec_cropped = spec_uint8[start_H:start_H + target_H, start_W:start_W + target_W]

        # Pad if smaller
        padded_img = np.zeros((224, 224), dtype=np.uint8)
        padded_img[:target_H, :target_W] = spec_cropped

        # Convert to RGB and resize to [3, 224, 224]
        img = Image.fromarray(padded_img).convert("RGB")
        img_np = np.asarray(img).transpose(2, 0, 1) / 255.0  # [3, 224, 224]

        # Normalize using ImageNet stats
        mean = np.array([0.485, 0.456, 0.406])[:, None, None]
        std = np.array([0.229, 0.224, 0.225])[:, None, None]
        img_np = (img_np - mean) / std

        return {
            "input_values": torch.tensor(img_np, dtype=torch.float32),
            "labels": torch.tensor(label, dtype=torch.long)
        }

def simple_collate_fn(batch):
    specs = [x["input_values"] for x in batch]
    labels = [x["labels"] for x in batch]
    return {
        "input_values": torch.stack(specs),
        "labels": torch.stack(labels)
    }


In [ ]:
# Imbalance class techniques: 
# Oversampling

count_0 = 22800
count_1 = 2580
total = count_0 + count_1

# Inverse frequency
class_weights = {
    0: total / count_0, 
    1: total / count_1   
}

# Get all labels from the dataset
labels = train_dataset.get_labels()

# Assign sample weights
sample_weights = [class_weights[label] for label in labels]


# Create the sampler
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=45600,
    replacement=True
)

# Create the DataLoader
train_dataloader = DataLoader(
    train_dataset,
    batch_size=8,
    sampler=sampler,
    collate_fn=simple_collate_fn
)

In [4]:
train_dataset = ConvNextFlacDataset(
    csv_path="/root/autodl-fs/ASVspoof2019.LA.cm.train.trn.txt",
    audio_dir="/root/autodl-fs/ASVspoof2019_LA_train/flac",
)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=simple_collate_fn)


for batch in train_dataloader:
    print(batch["input_values"].shape)  # [B, 3, 224, 224]
    print(batch["labels"])
    break


torch.Size([8, 3, 224, 224])
tensor([1, 0, 0, 0, 0, 0, 0, 0])


In [5]:
dev_dataset = ConvNextFlacDataset(
    csv_path="/root/autodl-fs/ASVspoof2019.LA.cm.dev.trl.txt",
    audio_dir="/root/autodl-fs/ASVspoof2019_LA_dev/flac",
)

dev_dataloader = DataLoader(dev_dataset, batch_size=8, shuffle=True, collate_fn=simple_collate_fn)

for batch in dev_dataloader:
    print(batch["input_values"].shape)  # [B, 3, 224, 224]
    print(batch["labels"])
    break


torch.Size([8, 3, 224, 224])
tensor([0, 0, 0, 0, 0, 0, 1, 0])


In [ ]:
eval_dataset = ConvNextFlacDataset(
    csv_path="/root/autodl-fs/ASVspoof2019.LA.cm.eval.trl.txt",
    audio_dir="/root/autodl-tmp/ASVspoof2019_LA_eval/flac",
)

eval_dataloader = DataLoader(eval_dataset, batch_size=8, shuffle=True, collate_fn=simple_collate_fn)

for batch in eval_dataloader:
    print(batch["input_values"].shape)  # [B, 3, 224, 224]
    print(batch["labels"])
    break


    

torch.Size([8, 3, 224, 224])
tensor([0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:


# Model Setup, using download path from huggingface
model_path = "..."

model = ConvNextForImageClassification.from_pretrained(
    model_path,
    num_labels=2,
    ignore_mismatched_sizes=True
).to(device)

optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)

# Imbalance class mitigation techniques 2:
# weighted loss
# comput weight
count_0 = 22800  # spoof
count_1 = 2580   # bonafide
total = count_0 + count_1

# Inverse frequency weights
weight_0 = total / count_0   
weight_1 = total / count_1  

# Convert to tensor
class_weights = torch.tensor([weight_0, weight_1], dtype=torch.float).to(device)

# Define weighted loss
criterion = nn.CrossEntropyLoss(weight=class_weights)


Some weights of ConvNextForImageClassification were not initialized from the model checkpoint at /root/autodl-tmp/convnext-tiny-224-copy-1/convnext-tiny-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

def compute_eer(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label=1)
    fnr = 1 - tpr
    eer_idx = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_idx] + fnr[eer_idx]) / 2
    return eer

def train(model, dataloader):
    model.train()
    total_loss, correct, total = 0, 0, 0
    y_true, y_score, y_pred = [], [], []

    for batch in tqdm(dataloader, desc="Training"):
        x = batch["input_values"].to(device)
        y = batch["labels"].to(device)

        out = model(pixel_values=x)
        logits = out.logits
        loss = criterion(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

        probs = torch.softmax(logits, dim=1)[:, 1]  # class 1 (bonafide)
        y_true.extend(y.cpu().numpy())
        y_score.extend(probs.detach().cpu().numpy())
        y_pred.extend(preds.detach().cpu().numpy())

    scheduler.step()
    acc = correct / total
    eer = compute_eer(y_true, y_score)
    auc = roc_auc_score(y_true, y_score)
    f1 = f1_score(y_true, y_pred)
    return total_loss / total, acc, eer, auc, f1

def evaluate(model, dataloader):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    y_true, y_score, y_pred = [], [], []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            x = batch["input_values"].to(device)
            y = batch["labels"].to(device)

            out = model(pixel_values=x)
            logits = out.logits
            loss = criterion(logits, y)

            total_loss += loss.item() * x.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

            probs = torch.softmax(logits, dim=1)[:, 1]
            y_true.extend(y.cpu().numpy())
            y_score.extend(probs.cpu().numpy())
            y_pred.extend(preds.detach().cpu().numpy())

    acc = correct / total
    eer = compute_eer(y_true, y_score)
    auc = roc_auc_score(y_true, y_score)
    f1 = f1_score(y_true, y_pred)
    return total_loss / total, acc, eer, auc, f1


In [ ]:
# Freeze layers (Linear Probing)

for param in model.convnext.parameters():
    param.requires_grad = False

for name, param in model.named_parameters():
    print(name, param.requires_grad)
    
# Partial tuning

# for i in range(3):
#     for param in model.convnext.encoder.stages[i].parameters():
#         param.requires_grad = False
# for param in model.convnext.encoder.stages[3].parameters():
#     param.requires_grad = True

# for name, param in model.named_parameters():
#     print(name, param.requires_grad)

# Full tuning

convnext.embeddings.patch_embeddings.weight False
convnext.embeddings.patch_embeddings.bias False
convnext.embeddings.layernorm.weight False
convnext.embeddings.layernorm.bias False
convnext.encoder.stages.0.layers.0.layer_scale_parameter False
convnext.encoder.stages.0.layers.0.dwconv.weight False
convnext.encoder.stages.0.layers.0.dwconv.bias False
convnext.encoder.stages.0.layers.0.layernorm.weight False
convnext.encoder.stages.0.layers.0.layernorm.bias False
convnext.encoder.stages.0.layers.0.pwconv1.weight False
convnext.encoder.stages.0.layers.0.pwconv1.bias False
convnext.encoder.stages.0.layers.0.pwconv2.weight False
convnext.encoder.stages.0.layers.0.pwconv2.bias False
convnext.encoder.stages.0.layers.1.layer_scale_parameter False
convnext.encoder.stages.0.layers.1.dwconv.weight False
convnext.encoder.stages.0.layers.1.dwconv.bias False
convnext.encoder.stages.0.layers.1.layernorm.weight False
convnext.encoder.stages.0.layers.1.layernorm.bias False
convnext.encoder.stages.0.la

In [43]:

epochs = 3
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss, train_acc, train_eer, train_auc, train_f1 = train(model, train_dataloader)
    print(f"Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.6f} | Train EER: {train_eer:.6f} | Train AUC: {train_auc:.6f} | Train F1: {train_f1:.6f}" )

    dev_loss, dev_acc, dev_eer, dev_auc, dev_f1 = evaluate(model, dev_dataloader)
    print(f"Dev   Loss: {dev_loss:.6f} | Dev   Acc: {dev_acc:.6f} | Dev EER: {dev_eer:.6f} | Dev AUC: {dev_auc:.6f} | Dev F1: {dev_f1:.6f}")

    model.eval()
    eval_loss, eval_acc, eval_eer, eval_auc, eval_f1 = evaluate(model, eval_dataloader)
    print(f"Eval  Loss: {eval_loss:.6f} | Eval   Acc: {eval_acc:.6f} | Eval EER: {eval_eer:.6f} | Eval AUC: {eval_auc:.6f} | Eval F1: {eval_f1:.6f}")



Epoch 1/3


Training: 100%|██████████| 3173/3173 [12:16<00:00,  4.31it/s]


Train Loss: 0.559229 | Train Acc: 0.831954 | Train EER: 0.311973 | Train AUC: 0.768875 | Train F1: 0.257615


Evaluating: 100%|██████████| 3106/3106 [12:06<00:00,  4.28it/s]


Dev   Loss: 0.512924 | Dev   Acc: 0.759499 | Dev EER: 0.267644 | Dev AUC: 0.816671 | Dev F1: 0.356350


...

Train Loss: 0.480218 | Train Acc: 0.796612 | Train EER: 0.253020 | Train AUC: 0.834658 | Train F1: 0.362559


Evaluating: 100%|██████████| 3106/3106 [12:33<00:00,  4.12it/s]


Dev   Loss: 0.473084 | Dev   Acc: 0.786508 | Dev EER: 0.239303 | Dev AUC: 0.844161 | Dev F1: 0.393829


Evaluating: 100%|██████████| 8905/8905 [23:14<00:00,  6.39it/s]


Eval  Loss: 0.506473 | Eval   Acc: 0.781855 | Eval EER: 0.267418 | Eval AUC: 0.819617 | Eval F1: 0.372223


In [44]:

save_dir = "/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl"
os.makedirs(save_dir, exist_ok=True)
model_path = "/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/3epoches_weights.pth"
torch.save(model.state_dict(), model_path)

print(f"Model parameters saved to {model_path}")

Model parameters saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/3epoches_weights.pth


In [11]:

model.load_state_dict(torch.load("/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/3epoches_weights.pth"))
    
epochs = 5
for epoch in range(3, epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss, train_acc, train_eer, train_auc, train_f1 = train(model, train_dataloader)
    print(f"Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.6f} | Train EER: {train_eer:.6f} | Train AUC: {train_auc:.6f} | Train F1: {train_f1:.6f}" )

    dev_loss, dev_acc, dev_eer, dev_auc, dev_f1 = evaluate(model, dev_dataloader)
    print(f"Dev   Loss: {dev_loss:.6f} | Dev   Acc: {dev_acc:.6f} | Dev EER: {dev_eer:.6f} | Dev AUC: {dev_auc:.6f} | Dev F1: {dev_f1:.6f}")

    model.eval()
    eval_loss, eval_acc, eval_eer, eval_auc, eval_f1 = evaluate(model, eval_dataloader)
    print(f"Eval  Loss: {eval_loss:.6f} | Eval   Acc: {eval_acc:.6f} | Eval EER: {eval_eer:.6f} | Eval AUC: {eval_auc:.6f} | Eval F1: {eval_f1:.6f}")




Epoch 4/5


Training: 100%|██████████| 3173/3173 [11:01<00:00,  4.79it/s]


Train Loss: 0.463788 | Train Acc: 0.806462 | Train EER: 0.238462 | Train AUC: 0.847271 | Train F1: 0.388903


Evaluating: 100%|██████████| 3106/3106 [12:31<00:00,  4.13it/s]


Dev   Loss: 0.457524 | Dev   Acc: 0.801240 | Dev EER: 0.227843 | Dev AUC: 0.856359 | Dev F1: 0.407203


...

Train Loss: 0.449006 | Train Acc: 0.815288 | Train EER: 0.228270 | Train AUC: 0.861361 | Train F1: 0.414146


Evaluating: 100%|██████████| 3106/3106 [11:52<00:00,  4.36it/s]


Dev   Loss: 0.446245 | Dev   Acc: 0.816374 | Dev EER: 0.220218 | Dev AUC: 0.865154 | Dev F1: 0.423407


Evaluating: 100%|██████████| 8905/8905 [30:36<00:00,  4.85it/s]


Eval  Loss: 0.478241 | Eval   Acc: 0.812317 | Eval EER: 0.247161 | Eval AUC: 0.843168 | Eval F1: 0.406411


In [12]:
model_path = "/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/5epoches_weights.pth"

torch.save(model.state_dict(), model_path)

print(f"Model parameters saved to {model_path}")

Model parameters saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/5epoches_weights.pth


In [10]:
model.load_state_dict(torch.load("/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/5epoches_weights.pth"))
save_dir = "/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/"

epochs = 10
for epoch in range(5, epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss, train_acc, train_eer, train_auc, train_f1 = train(model, train_dataloader)
    print(f"Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.6f} | Train EER: {train_eer:.6f} | Train AUC: {train_auc:.6f} | Train F1: {train_f1:.6f}" )

    dev_loss, dev_acc, dev_eer, dev_auc, dev_f1 = evaluate(model, dev_dataloader)
    print(f"Dev   Loss: {dev_loss:.6f} | Dev   Acc: {dev_acc:.6f} | Dev EER: {dev_eer:.6f} | Dev AUC: {dev_auc:.6f} | Dev F1: {dev_f1:.6f}")

    
    save_path = os.path.join(save_dir, f"epoch_{epoch+1}.pt")
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")




Epoch 6/10


Training: 100%|██████████| 3173/3173 [19:07<00:00,  2.77it/s]


Train Loss: 0.435528 | Train Acc: 0.821552 | Train EER: 0.214755 | Train AUC: 0.872294 | Train F1: 0.431673


Evaluating: 100%|██████████| 3106/3106 [19:14<00:00,  2.69it/s]


Dev   Loss: 0.435469 | Dev   Acc: 0.844429 | Dev EER: 0.213827 | Dev AUC: 0.872202 | Dev F1: 0.432702
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_6.pt

Epoch 7/10


...

Dev   Loss: 0.424683 | Dev   Acc: 0.839961 | Dev EER: 0.203281 | Dev AUC: 0.882058 | Dev F1: 0.460369
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_9.pt

Epoch 10/10


Training: 100%|██████████| 3173/3173 [19:43<00:00,  2.68it/s]


Train Loss: 0.409119 | Train Acc: 0.844484 | Train EER: 0.192590 | Train AUC: 0.894699 | Train F1: 0.478117


Evaluating: 100%|██████████| 3106/3106 [19:17<00:00,  2.68it/s]


Dev   Loss: 0.422112 | Dev   Acc: 0.822774 | Dev EER: 0.202950 | Dev AUC: 0.882986 | Dev F1: 0.456755
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_10.pt


In [12]:
model.eval()
eval_loss, eval_acc, eval_eer, eval_auc, eval_f1 = evaluate(model, eval_dataloader)
print(f"Eval  Loss: {eval_loss:.6f} | Eval   Acc: {eval_acc:.6f} | Eval EER: {eval_eer:.6f} | Eval AUC: {eval_auc:.6f} | Eval F1: {eval_f1:.6f}")



Evaluating: 100%|██████████| 8905/8905 [45:24<00:00,  3.27it/s]


Eval  Loss: 0.460901 | Eval   Acc: 0.804526 | Eval EER: 0.228157 | Eval AUC: 0.860473 | Eval F1: 0.427402


In [13]:

save_dir = "/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/"

epochs = 15
for epoch in range(10, epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss, train_acc, train_eer, train_auc, train_f1 = train(model, train_dataloader)
    print(f"Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.6f} | Train EER: {train_eer:.6f} | Train AUC: {train_auc:.6f} | Train F1: {train_f1:.6f}" )

    dev_loss, dev_acc, dev_eer, dev_auc, dev_f1 = evaluate(model, dev_dataloader)
    print(f"Dev   Loss: {dev_loss:.6f} | Dev   Acc: {dev_acc:.6f} | Dev EER: {dev_eer:.6f} | Dev AUC: {dev_auc:.6f} | Dev F1: {dev_f1:.6f}")

    
    save_path = os.path.join(save_dir, f"epoch_{epoch+1}.pt")
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")


Epoch 11/15


Training: 100%|██████████| 3173/3173 [16:50<00:00,  3.14it/s]


Train Loss: 0.405295 | Train Acc: 0.841056 | Train EER: 0.191113 | Train AUC: 0.895736 | Train F1: 0.477867


Evaluating: 100%|██████████| 3106/3106 [11:11<00:00,  4.63it/s]


Dev   Loss: 0.421628 | Dev   Acc: 0.829134 | Dev EER: 0.201313 | Dev AUC: 0.883985 | Dev F1: 0.461773
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_11.pt

Epoch 12/15


...

Dev   Loss: 0.417712 | Dev   Acc: 0.836178 | Dev EER: 0.200982 | Dev AUC: 0.884993 | Dev F1: 0.466019
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_14.pt

Epoch 15/15


Training: 100%|██████████| 3173/3173 [03:04<00:00, 17.20it/s]


Train Loss: 0.398734 | Train Acc: 0.844011 | Train EER: 0.188550 | Train AUC: 0.898247 | Train F1: 0.482010


Evaluating: 100%|██████████| 3106/3106 [03:00<00:00, 17.21it/s]


Dev   Loss: 0.418265 | Dev   Acc: 0.840968 | Dev EER: 0.200517 | Dev AUC: 0.885216 | Dev F1: 0.466297
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_15.pt


In [14]:

save_dir = "/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/"

epochs = 20
for epoch in range(15, epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss, train_acc, train_eer, train_auc, train_f1 = train(model, train_dataloader)
    print(f"Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.6f} | Train EER: {train_eer:.6f} | Train AUC: {train_auc:.6f} | Train F1: {train_f1:.6f}" )

    dev_loss, dev_acc, dev_eer, dev_auc, dev_f1 = evaluate(model, dev_dataloader)
    print(f"Dev   Loss: {dev_loss:.6f} | Dev   Acc: {dev_acc:.6f} | Dev EER: {dev_eer:.6f} | Dev AUC: {dev_auc:.6f} | Dev F1: {dev_f1:.6f}")

    
    save_path = os.path.join(save_dir, f"epoch_{epoch+1}.pt")
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")


Epoch 16/20


Training: 100%|██████████| 3173/3173 [10:20<00:00,  5.11it/s]


Train Loss: 0.400060 | Train Acc: 0.848700 | Train EER: 0.188353 | Train AUC: 0.898574 | Train F1: 0.487042


Evaluating: 100%|██████████| 3106/3106 [11:47<00:00,  4.39it/s]


Dev   Loss: 0.418513 | Dev   Acc: 0.839599 | Dev EER: 0.200517 | Dev AUC: 0.885305 | Dev F1: 0.466604
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_16.pt

Epoch 17/20


...

Dev   Loss: 0.416290 | Dev   Acc: 0.839559 | Dev EER: 0.200141 | Dev AUC: 0.885509 | Dev F1: 0.467112
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_19.pt

Epoch 20/20


Training: 100%|██████████| 3173/3173 [12:32<00:00,  4.22it/s]


Train Loss: 0.400223 | Train Acc: 0.845508 | Train EER: 0.187658 | Train AUC: 0.899000 | Train F1: 0.485501


Evaluating: 100%|██████████| 3106/3106 [12:41<00:00,  4.08it/s]


Dev   Loss: 0.416095 | Dev   Acc: 0.839237 | Dev EER: 0.200119 | Dev AUC: 0.885536 | Dev F1: 0.466898
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_20.pt


In [10]:
model.load_state_dict(torch.load("/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_20.pt"))

save_dir = "/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/"

epochs = 25
for epoch in range(20, epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss, train_acc, train_eer, train_auc, train_f1 = train(model, train_dataloader)
    print(f"Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.6f} | Train EER: {train_eer:.6f} | Train AUC: {train_auc:.6f} | Train F1: {train_f1:.6f}" )

    save_path = os.path.join(save_dir, f"epoch_{epoch+1}.pt")
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")
    
    dev_loss, dev_acc, dev_eer, dev_auc, dev_f1 = evaluate(model, dev_dataloader)
    print(f"Dev   Loss: {dev_loss:.6f} | Dev   Acc: {dev_acc:.6f} | Dev EER: {dev_eer:.6f} | Dev AUC: {dev_auc:.6f} | Dev F1: {dev_f1:.6f}")

    
    


Epoch 21/25


Training: 100%|██████████| 3173/3173 [10:29<00:00,  5.04it/s]


Train Loss: 0.397843 | Train Acc: 0.847636 | Train EER: 0.187483 | Train AUC: 0.900736 | Train F1: 0.490581
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_21.pt


Evaluating: 100%|██████████| 3106/3106 [09:19<00:00,  5.56it/s]


Dev   Loss: 0.413916 | Dev   Acc: 0.851191 | Dev EER: 0.196250 | Dev AUC: 0.888006 | Dev F1: 0.476568

Epoch 22/25


...

Dev   Loss: 0.404882 | Dev   Acc: 0.848495 | Dev EER: 0.190834 | Dev AUC: 0.892924 | Dev F1: 0.486634

Epoch 25/25


Training: 100%|██████████| 3173/3173 [11:42<00:00,  4.52it/s]


Train Loss: 0.379236 | Train Acc: 0.857132 | Train EER: 0.175207 | Train AUC: 0.911426 | Train F1: 0.515111
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_25.pt


Evaluating: 100%|██████████| 3106/3106 [09:30<00:00,  5.44it/s]

Dev   Loss: 0.403247 | Dev   Acc: 0.850064 | Dev EER: 0.190369 | Dev AUC: 0.893325 | Dev F1: 0.487832


In [11]:
eval_loss, eval_acc, eval_eer, eval_auc, eval_f1 = evaluate(model, eval_dataloader)
print(f"Eval  Loss: {eval_loss:.6f} | Eval   Acc: {eval_acc:.6f} | Eval EER: {eval_eer:.6f} | Eval AUC: {eval_auc:.6f} | Eval F1: {eval_f1:.6f}")


Evaluating: 100%|██████████| 8905/8905 [26:23<00:00,  5.62it/s]


Eval  Loss: 0.442991 | Eval   Acc: 0.827590 | Eval EER: 0.217663 | Eval AUC: 0.869955 | Eval F1: 0.451354


In [13]:

save_dir = "/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/"

epochs = 30
for epoch in range(25, epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss, train_acc, train_eer, train_auc, train_f1 = train(model, train_dataloader)
    print(f"Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.6f} | Train EER: {train_eer:.6f} | Train AUC: {train_auc:.6f} | Train F1: {train_f1:.6f}" )

    save_path = os.path.join(save_dir, f"epoch_{epoch+1}.pt")
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")
    
    dev_loss, dev_acc, dev_eer, dev_auc, dev_f1 = evaluate(model, dev_dataloader)
    print(f"Dev   Loss: {dev_loss:.6f} | Dev   Acc: {dev_acc:.6f} | Dev EER: {dev_eer:.6f} | Dev AUC: {dev_auc:.6f} | Dev F1: {dev_f1:.6f}")

eval_loss, eval_acc, eval_eer, eval_auc, eval_f1 = evaluate(model, eval_dataloader)
print(f"Eval  Loss: {eval_loss:.6f} | Eval   Acc: {eval_acc:.6f} | Eval EER: {eval_eer:.6f} | Eval AUC: {eval_auc:.6f} | Eval F1: {eval_f1:.6f}")

    


Epoch 26/30


Training: 100%|██████████| 3173/3173 [14:50<00:00,  3.56it/s]


Train Loss: 0.379757 | Train Acc: 0.858629 | Train EER: 0.172838 | Train AUC: 0.912022 | Train F1: 0.517223
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_26.pt


Evaluating: 100%|██████████| 3106/3106 [15:37<00:00,  3.31it/s]


Dev   Loss: 0.401606 | Dev   Acc: 0.853083 | Dev EER: 0.189618 | Dev AUC: 0.893789 | Dev F1: 0.488509

Epoch 27/30


...

Train Loss: 0.376998 | Train Acc: 0.858392 | Train EER: 0.171317 | Train AUC: 0.913344 | Train F1: 0.519776
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_30.pt


Evaluating: 100%|██████████| 3106/3106 [10:21<00:00,  5.00it/s]


Dev   Loss: 0.401675 | Dev   Acc: 0.851956 | Dev EER: 0.189640 | Dev AUC: 0.894372 | Dev F1: 0.490441


Evaluating: 100%|██████████| 8905/8905 [25:51<00:00,  5.74it/s]


Eval  Loss: 0.438036 | Eval   Acc: 0.829162 | Eval EER: 0.216318 | Eval AUC: 0.871094 | Eval F1: 0.453917


In [10]:

save_dir = "/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/"
model.load_state_dict(torch.load("/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_30.pt"))

epochs = 40
for epoch in range(30, epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss, train_acc, train_eer, train_auc, train_f1 = train(model, train_dataloader)
    print(f"Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.6f} | Train EER: {train_eer:.6f} | Train AUC: {train_auc:.6f} | Train F1: {train_f1:.6f}" )

    save_path = os.path.join(save_dir, f"epoch_{epoch+1}.pt")
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")
    
    dev_loss, dev_acc, dev_eer, dev_auc, dev_f1 = evaluate(model, dev_dataloader)
    print(f"Dev   Loss: {dev_loss:.6f} | Dev   Acc: {dev_acc:.6f} | Dev EER: {dev_eer:.6f} | Dev AUC: {dev_auc:.6f} | Dev F1: {dev_f1:.6f}")

eval_loss, eval_acc, eval_eer, eval_auc, eval_f1 = evaluate(model, eval_dataloader)
print(f"Eval  Loss: {eval_loss:.6f} | Eval   Acc: {eval_acc:.6f} | Eval EER: {eval_eer:.6f} | Eval AUC: {eval_auc:.6f} | Eval F1: {eval_f1:.6f}")

    


Epoch 31/40


Training: 100%|██████████| 3173/3173 [10:44<00:00,  4.92it/s]


Train Loss: 0.375912 | Train Acc: 0.859614 | Train EER: 0.169862 | Train AUC: 0.914046 | Train F1: 0.523089
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_31.pt


Evaluating: 100%|██████████| 3106/3106 [11:58<00:00,  4.33it/s]


Dev   Loss: 0.397271 | Dev   Acc: 0.846080 | Dev EER: 0.187672 | Dev AUC: 0.895804 | Dev F1: 0.488360

Epoch 32/40


...

Train Loss: 0.359059 | Train Acc: 0.869346 | Train EER: 0.159670 | Train AUC: 0.922214 | Train F1: 0.543376
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_40.pt


Evaluating: 100%|██████████| 3106/3106 [12:33<00:00,  4.12it/s]


Dev   Loss: 0.392337 | Dev   Acc: 0.856867 | Dev EER: 0.183271 | Dev AUC: 0.899605 | Dev F1: 0.502240


Evaluating: 100%|██████████| 8905/8905 [26:50<00:00,  5.53it/s]


Eval  Loss: 0.431238 | Eval   Acc: 0.829302 | Eval EER: 0.210744 | Eval AUC: 0.876300 | Eval F1: 0.461470


In [11]:

save_dir = "/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/"


epochs = 50
for epoch in range(40, epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss, train_acc, train_eer, train_auc, train_f1 = train(model, train_dataloader)
    print(f"Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.6f} | Train EER: {train_eer:.6f} | Train AUC: {train_auc:.6f} | Train F1: {train_f1:.6f}" )

    save_path = os.path.join(save_dir, f"epoch_{epoch+1}.pt")
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")
    
    dev_loss, dev_acc, dev_eer, dev_auc, dev_f1 = evaluate(model, dev_dataloader)
    print(f"Dev   Loss: {dev_loss:.6f} | Dev   Acc: {dev_acc:.6f} | Dev EER: {dev_eer:.6f} | Dev AUC: {dev_auc:.6f} | Dev F1: {dev_f1:.6f}")

eval_loss, eval_acc, eval_eer, eval_auc, eval_f1 = evaluate(model, eval_dataloader)
print(f"Eval  Loss: {eval_loss:.6f} | Eval   Acc: {eval_acc:.6f} | Eval EER: {eval_eer:.6f} | Eval AUC: {eval_auc:.6f} | Eval F1: {eval_f1:.6f}")

    


Epoch 41/50


Training: 100%|██████████| 3173/3173 [08:00<00:00,  6.60it/s]


Train Loss: 0.359050 | Train Acc: 0.868558 | Train EER: 0.159670 | Train AUC: 0.922349 | Train F1: 0.544013
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_41.pt


Evaluating: 100%|██████████| 3106/3106 [05:52<00:00,  8.81it/s]


Dev   Loss: 0.392145 | Dev   Acc: 0.856585 | Dev EER: 0.183226 | Dev AUC: 0.899646 | Dev F1: 0.502166

Epoch 42/50


...

Train Loss: 0.358296 | Train Acc: 0.869070 | Train EER: 0.158931 | Train AUC: 0.922530 | Train F1: 0.544857
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_50.pt


Evaluating: 100%|██████████| 3106/3106 [05:54<00:00,  8.75it/s]


Dev   Loss: 0.393551 | Dev   Acc: 0.857833 | Dev EER: 0.183339 | Dev AUC: 0.899740 | Dev F1: 0.503375


Evaluating: 100%|██████████| 8905/8905 [14:29<00:00, 10.24it/s]


Eval  Loss: 0.431750 | Eval   Acc: 0.830229 | Eval EER: 0.210737 | Eval AUC: 0.876351 | Eval F1: 0.462202


In [10]:

save_dir = "/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/"
model.load_state_dict(torch.load("/root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_50.pt"))


epochs = 60
for epoch in range(50, epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss, train_acc, train_eer, train_auc, train_f1 = train(model, train_dataloader)
    print(f"Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.6f} | Train EER: {train_eer:.6f} | Train AUC: {train_auc:.6f} | Train F1: {train_f1:.6f}" )

    save_path = os.path.join(save_dir, f"epoch_{epoch+1}.pt")
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")
    
dev_loss, dev_acc, dev_eer, dev_auc, dev_f1 = evaluate(model, dev_dataloader)
print(f"Dev   Loss: {dev_loss:.6f} | Dev   Acc: {dev_acc:.6f} | Dev EER: {dev_eer:.6f} | Dev AUC: {dev_auc:.6f} | Dev F1: {dev_f1:.6f}")

eval_loss, eval_acc, eval_eer, eval_auc, eval_f1 = evaluate(model, eval_dataloader)
print(f"Eval  Loss: {eval_loss:.6f} | Eval   Acc: {eval_acc:.6f} | Eval EER: {eval_eer:.6f} | Eval AUC: {eval_auc:.6f} | Eval F1: {eval_f1:.6f}")



Epoch 51/60


Training: 100%|██████████| 3173/3173 [13:55<00:00,  3.80it/s]


Train Loss: 0.359231 | Train Acc: 0.870449 | Train EER: 0.157933 | Train AUC: 0.922846 | Train F1: 0.546107
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_51.pt

Epoch 52/60


Training: 100%|██████████| 3173/3173 [09:26<00:00,  5.60it/s]


Train Loss: 0.352349 | Train Acc: 0.869858 | Train EER: 0.156953 | Train AUC: 0.923757 | Train F1: 0.546976
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_52.pt

Epoch 53/60


...

Train Loss: 0.344827 | Train Acc: 0.876005 | Train EER: 0.150413 | Train AUC: 0.928227 | Train F1: 0.560904
Model saved to /root/autodl-fs/convnext_tiny_fine_tuning/lp_trun_wl/epoch_60.pt


Evaluating: 100%|██████████| 3106/3106 [14:22<00:00,  3.60it/s]


Dev   Loss: 0.385608 | Dev   Acc: 0.864877 | Dev EER: 0.180642 | Dev AUC: 0.903188 | Dev F1: 0.513408


Evaluating: 100%|██████████| 8905/8905 [28:04<00:00,  5.29it/s]


Eval  Loss: 0.422554 | Eval   Acc: 0.835788 | Eval EER: 0.206754 | Eval AUC: 0.879867 | Eval F1: 0.471110
